In [1]:

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
import pandas as pd
df = pd.read_csv("galaxy7.csv",sep='delimiter', header=None)
#df

<ipython-input-2-44f9f2813ce5>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("galaxy7.csv",sep='delimiter', header=None)


In [3]:
df.columns =['Review']
#df

In [4]:
df = df.dropna()
df

,Review
0,yay ! i got burns now :D
1,if they can bring back this design with update...
2,"""Hehe think about a note 7 on a plane now lol ..."
3,bomb review
4,MKBHD dodged a bullet here reviewing this gren...
...,...
5707,FIRST!
5708,First
5709,wooooo
5710,Watching....


In [5]:
import re
import string
import csv
df['Review'] = df['Review'].str.replace(r'[^\w\s]+', '')
#df

<ipython-input-5-e8410bf56ef0>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Review'] = df['Review'].str.replace(r'[^\w\s]+', '')


In [6]:
df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
#df.to_csv('xyz.csv', sep='\t', index=False,header=True)

,Review
0,yay i got burns now D
1,if they can bring back this design with update...
2,Hehe think about a note 7 on a plane now lol a...
3,bomb review
4,MKBHD dodged a bullet here reviewing this grenade
...,...
5707,FIRST
5708,First
5709,wooooo
5710,Watching


In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["Review"]]
df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["Review"]]
df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["Review"]]
df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in df["Review"]]
#df

In [19]:
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
#df
#df.to_csv('Bioderma.csv',index=False)

In [20]:
df.to_csv('galaxy7_labeled.csv',index=False)

In [21]:
print(df["Sentiment"].value_counts())

Neutral     2604
Positive    2190
Negative     918
Name: Sentiment, dtype: int64


In [22]:
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
data = pd.read_csv('galaxy7_labeled.csv')

In [23]:
data.drop(['Positive','Negative','Neutral','Compound'],axis=1,inplace=True)
data

,Review,Sentiment
0,yay i got burns now D,Positive
1,if they can bring back this design with update...,Positive
2,Hehe think about a note 7 on a plane now lol a...,Positive
3,bomb review,Negative
4,MKBHD dodged a bullet here reviewing this grenade,Neutral
...,...,...
5707,FIRST,Neutral
5708,First,Neutral
5709,wooooo,Neutral
5710,Watching,Neutral


In [24]:
data.drop(data[(data['Sentiment'] =='Neutral')].index, inplace=True)


In [25]:
data

,Review,Sentiment
0,yay i got burns now D,Positive
1,if they can bring back this design with update...,Positive
2,Hehe think about a note 7 on a plane now lol a...,Positive
3,bomb review,Negative
7,it bomb,Negative
...,...,...
5692,8th fuck,Negative
5693,Amazing how slick your videos always are,Positive
5698,8 views 9 likes,Positive
5701,awesome,Positive


In [16]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['Review'] = data.Review.apply(lemmatize_text)

In [17]:
data

,Review,Sentiment
0,I added this to my daily routine after a bad a...,Positive
2,Apparently mineral sunscreen tends to leave a ...,Positive
3,The sunscreen is good but it leaf a white cast...,Positive
4,I like the light tint and it seems to be a ver...,Positive
5,I use this sunscreen kitesurfing in Mexico It ...,Positive
...,...,...
1789,This brand is always really great with sensiti...,Positive
1790,This stuff is pretty good I use the version th...,Positive
1791,I am a fan of this brand I use a lot of their ...,Positive
1792,Dont get me wrong I love Cerave product I had ...,Positive


In [18]:
reviews = data['Review'].values
labels = data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [19]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [20]:
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names()
X = X.toarray()
word_counts = {}
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

In [21]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

In [22]:
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data

In [23]:
def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

In [24]:
from nltk.tokenize import sent_tokenize,word_tokenize
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result

In [25]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_sentences,train_labels,labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
#print("Prediction Results")
#print(pred)
accuaracy=accuracy_score(test_labels,pred)*100
print("Accuracy : ", accuaracy,"%")

Accuracy :  66.16541353383458 %


In [54]:
#productname="erave"
#f = open("Accuracies.txt", "a")
#f.write(productname+ " " + str(accuaracy) )
#f.close()

In [28]:
data2=data
data2.drop(['Sentiment'],axis=1,inplace=True)
data2

,Review
0,I added this to my daily routine after a bad a...
2,Apparently mineral sunscreen tends to leave a ...
3,The sunscreen is good but it leaf a white cast...
4,I like the light tint and it seems to be a ver...
5,I use this sunscreen kitesurfing in Mexico It ...
...,...
1789,This brand is always really great with sensiti...
1790,This stuff is pretty good I use the version th...
1791,I am a fan of this brand I use a lot of their ...
1792,Dont get me wrong I love Cerave product I had ...


In [31]:
import nltk.classify.util #calculates accuracy
from nltk.classify import NaiveBayesClassifier #imports the classifier Naive Bayes
from nltk.corpus import movie_reviews #imports movie reviews from nltk
from nltk.corpus import stopwords #imports stopwords from nltk
from nltk.corpus import wordnet #imports wordnet(lexical database for the english language) from nltk

In [33]:
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

#For each word, we create

In [34]:
words = word_tokenize(str(data2))
words = create_word_features(words)
classifier.classify(words)

NameError: name 'classifier' is not defined